In [1]:
# given a list of DCs, and a wrong repair(hand-picked), return the responsibility of the DCs

### 1. run holoclean and return wrongly repaired cells 

In [2]:
from examples.holoclean_repair_example_author import main
import psycopg2
import pandas as pd
import copy
pd.set_option('display.max_rows', None)

In [3]:
# connection to db 
conn = psycopg2.connect(dbname="holo", user="holocleanuser", password="abcd1234")
cols = ['aid','name','oid','organization']
cur = conn.cursor()
conn.autocommit=True

In [4]:
# drop preexisted repaired records 
select_old_repairs_q = """
SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_NAME LIKE 'author_repaired_%' AND TABLE_TYPE = 'BASE TABLE'
"""
cur.execute(select_old_repairs_q)


In [5]:
for records in cur.fetchall():
    drop_q = f"drop table if exists {records[0]}"
    cur.execute(drop_q)

In [6]:
main(initial_training=True)

09:57:16 - [DEBUG] - initiating session with parameters: {'db_user': 'holocleanuser', 'db_pwd': 'abcd1234', 'db_host': 'localhost', 'db_name': 'holo', 'threads': 1, 'timeout': 180000, 'seed': 45, 'learning_rate': 0.001, 'optimizer': 'adam', 'epochs': 10, 'weight_decay': 0.01, 'momentum': 0.0, 'batch_size': 1, 'weak_label_thresh': 0.99, 'domain_thresh_1': 0, 'domain_thresh_2': 0, 'max_domain': 10000, 'cor_strength': 0.6, 'nb_cor_strength': 0.8, 'feature_norm': False, 'weight_norm': False, 'estimator_epochs': 3, 'estimator_batch_size': 32, 'verbose': True, 'bias': False, 'print_fw': True, 'debug_mode': False}
09:57:16 - [ INFO] - Loaded 5012 rows with 25060 cells
09:57:19 - [DEBUG] - Time to create index: 0.00 secs
09:57:19 - [DEBUG] - Time to create index: 0.00 secs
09:57:19 - [DEBUG] - Time to create index: 0.00 secs
09:57:19 - [DEBUG] - Time to create index: 0.00 secs
09:57:19 - [ INFO] - DONE Loading holoclean_hauthor_100_errors.csv
09:57:19 - [DEBUG] - Time to load dataset: 1.25 sec

found our example
denominator = 18, domain_thresh_1=0, tau=0.0
top_cands: ['hayri onal', 'nail cadalli']


100%|██████████| 5012/5012 [00:03<00:00, 1301.92it/s]
09:57:25 - [DEBUG] - DONE generating initial set of domain values in 3.96
09:57:25 - [DEBUG] - training posterior model for estimating domain value probabilities...
09:57:25 - [DEBUG] - DONE training posterior model in 0.03s
09:57:25 - [DEBUG] - predicting domain value probabilities from posterior model...
09:57:25 - [DEBUG] - DONE predictions in 0.00 secs, re-constructing cell domain...
09:57:25 - [DEBUG] - re-assembling final cell domain table...
0it [00:00, ?it/s]
2490it [00:00, 24894.86it/s]:00<?, ?it/s]
5248it [00:00, 25642.95it/s][00:00<00:00, 28219.02it/s]
7909it [00:00, 25924.58it/s][00:00<00:00, 28048.51it/s]
 84%|████████▎ | 8385/10024 [00:00<00:00, 28024.54it/s]
10024it [00:00, 25575.01it/s][00:00<00:00, 27490.79it/s]
09:57:26 - [DEBUG] - DONE assembling cell domain table in 0.68s
09:57:26 - [ INFO] - number of (additional) weak labels assigned from posterior model: 59
09:57:26 - [DEBUG] - DONE generating domain and weak 

09:59:30 - [DEBUG] - Time to execute query with id 0: 0.00 secs
09:59:30 - [DEBUG] - Starting to execute query SELECT COUNT(*) FROM   (SELECT t2._tid_, t2._attribute_, t2._value_      FROM "author" as t1, "author_clean" as t2     WHERE t1._tid_ = t2._tid_       AND t2._attribute_ = 'name'       AND t1."name" != t2._value_ ) as errors, inf_values_dom as repairs WHERE errors._tid_ = repairs._tid_   AND errors._attribute_ = repairs.attribute   AND errors._value_ = repairs.rv_value with id 1
09:59:30 - [DEBUG] - Time to execute query with id 1: 0.00 secs
09:59:30 - [DEBUG] - Starting to execute query SELECT COUNT(*) FROM   (SELECT t2._tid_, t2._attribute_, t2._value_      FROM "author" as t1, "author_clean" as t2     WHERE t1._tid_ = t2._tid_       AND t2._attribute_ = 'oid'       AND t1."oid" != t2._value_ ) as errors, inf_values_dom as repairs WHERE errors._tid_ = repairs._tid_   AND errors._attribute_ = repairs.attribute   AND errors._value_ = repairs.rv_value with id 2
09:59:30 - [DEBU

In [7]:
### 2. run holoclean and return wrongly repaired ddcells 

In [8]:
union_sql = """
SELECT 'before_clean' AS type, * from  author
union all 
SELECT 'after_clean' AS type, * from author_repaired
order by _tid_
"""
df_union_before_and_after = pd.read_sql(union_sql, conn) 

In [9]:
df_union_before_and_after

,type,_tid_,aid,name,oid,organization
0,before_clean,0,1,ricardo del corazon grau-abalo,16771,universidad central marta abreu de la villas
1,after_clean,0,1,ricardo del corazon grau-abalo,16771,universidad central marta abreu de la villas
2,before_clean,1,2,nieves abalos,17015,universidad de granada
3,after_clean,1,2,nieves abalos,17015,universidad de granada
4,before_clean,2,4,erika abraham,14771,rheinisch westfalische technische hochschule a...
5,after_clean,2,4,erika abraham,14771,rheinisch westfalische technische hochschule a...
6,before_clean,3,5,bernardo m. abrego,6627,california state university northridge
7,after_clean,3,5,bernardo m. abrego,6627,california state university northridge
8,after_clean,4,8,gergely acs,21395,laboratory of cryptography and systems security
9,before_clean,4,8,gergely acs,21395,laboratory of cryptography and systems security


In [10]:
j = 0
for i in range(0,5000):
    if(j%100==0):
        print(j)
    j+=1
    df_row = pd.DataFrame(columns=['type']+cols)
    df_row.loc[0,'type'] = 'ground_truth'
    df_row.loc[0,'_tid_'] = i
    q = f"""
    SELECT * FROM author_clean WHERE _tid_={i} 
    """
    df_for_one_row = pd.read_sql(q,conn)
    for index, row in df_for_one_row.iterrows():
        df_row.loc[0, f"{row['_attribute_']}"] = row['_value_']
    df_union_before_and_after = df_union_before_and_after.append(df_row, ignore_index=True)

0


/home/jayli/Desktop/holo_env/lib/python3.6/site-packages/pandas/core/frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900


In [11]:
repaired_dict = {x:[] for x in cols}

In [12]:
# iterate this dataframe to find all wrong predictions and list them to choose
grouped = df_union_before_and_after.groupby('_tid_')
k = 0

for name, group in grouped:
    tid = pd.to_numeric(group.iloc[0]['_tid_'], downcast="integer")
    if(k%100==0):
        print(k)
    for c in cols:
        if(group[group['type']=='after_clean'][c].to_string(index=False)\
           != group[group['type']=='before_clean'][c].to_string(index=False)):
            repaired_dict[c].append(tid)
    k+=1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [13]:
repaired_dict

{'aid': [],
 'name': [],
 'oid': [],
 'organization': [28.0,
  48.0,
  64.0,
  86.0,
  105.0,
  111.0,
  114.0,
  128.0,
  133.0,
  162.0,
  169.0,
  172.0,
  218.0,
  266.0,
  304.0,
  313.0,
  319.0,
  340.0,
  367.0,
  389.0,
  397.0,
  422.0,
  459.0,
  462.0,
  482.0,
  489.0,
  496.0,
  509.0,
  524.0,
  537.0,
  542.0,
  614.0,
  618.0,
  660.0,
  672.0,
  678.0,
  696.0,
  749.0,
  777.0,
  780.0,
  783.0,
  806.0,
  813.0,
  830.0,
  837.0,
  841.0,
  864.0,
  878.0,
  941.0,
  944.0,
  988.0,
  992.0]}

In [14]:
repaired_hospital=df_union_before_and_after[df_union_before_and_after['_tid_'].isin(repaired_dict['name'])]

In [15]:
repaired_hospital.sort_values(by=['_tid_','type']).reset_index()

,index,_tid_,aid,name,oid,organization,type


In [16]:
wrong_dict = {x:[] for x in cols}

In [17]:
# iterate this dataframe to find all wrong predictions and list them to choose
grouped = df_union_before_and_after.groupby('_tid_')
k = 0

for name, group in grouped:
    tid = pd.to_numeric(group.iloc[0]['_tid_'], downcast="integer")
    if(k%100==0):
        print(k)
    for c in cols:
        if((group[group['type']=='after_clean'][c].to_string(index=False)\
           != group[group['type']=='ground_truth'][c].to_string(index=False)\
           ) and (group[group['type']=='after_clean'][c].to_string(index=False)\
           != group[group['type']=='before_clean'][c].to_string(index=False))):
            wrong_dict[c].append(tid)
    k+=1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000


In [18]:
wrong_dict

{'aid': [], 'name': [], 'oid': [], 'organization': []}

In [19]:
wrong_cities=df_union_before_and_after[df_union_before_and_after['_tid_'].isin(wrong_dict['HospitalName'])]

KeyError: 'HospitalName'

In [ ]:
wrong_cities

In [ ]:
wrong_cities.sort_values(by=['_tid_','type','HospitalOwner'], inplace=True)

In [ ]:
wrong_cities[['HospitalOwner','HospitalName', '_tid_', 'type']]

In [ ]:
wrong_dict

In [ ]:
wrong_dict['City']

In [ ]:
df_union_before_and_after

In [ ]:
wrong_cities

In [ ]:
conn.close()

### given an attribute and a tid (row id), find responsible rule(s)

In [ ]:
# import hc_responsibility
# hc_responsibility.rule_responsibility(attr_name='PhoneNumber', tid=381)

In [ ]:
# conn = psycopg2.connect(dbname="holo", user="holocleanuser", password="abcd1234")

### result after deleting 
    t1&t2&EQ(t1.HospitalName,t2.HospitalName)&IQ(t1.PhoneNumber,t2.PhoneNumber)
    AND 
    t1&t2&EQ(t1.HospitalName,t2.HospitalName)&IQ(t1.ZipCode,t2.ZipCode)

fixing using :

    t1&t2&EQ(t1.HospitalName,t2.HospitalName)&IQ(t1.ProviderNumber,t2.ProviderNumber)
    AND
    t1&t2&EQ(t1.HospitalName,t2.HospitalName)&EQ(t1.PhoneNumber,t2.PhoneNumber)&EQ(t1.HospitalOwner,t2.HospitalOwner)&IQ(t1.State,t2.State)

In [ ]:
# q_2 = """
# select * from hospital_repaired_2 where "Address1" = '2451 fillingim street'
# """
# df_repaired_2 = pd.read_sql(q_2, conn)

In [ ]:
# df_repaired_2

### result after deleting 
    t1&t2&EQ(t1.HospitalName,t2.HospitalName)&EQ(t1.PhoneNumber,t2.PhoneNumber)&EQ(t1.HospitalOwner,t2.HospitalOwner)&IQ(t1.State,t2.State)
    fixing using :
    t1&t2&EQ(t1.HospitalName,t2.HospitalName)&IQ(t1.PhoneNumber,t2.PhoneNumber)
    AND 
    t1&t2&EQ(t1.HospitalName,t2.HospitalName)&IQ(t1.ZipCode,t2.ZipCode)
    AND 
    t1&t2&EQ(t1.HospitalName,t2.HospitalName)&IQ(t1.ProviderNumber,t2.ProviderNumber)


In [ ]:
# q_6 = """
# select * from hospital_repaired_6 where "Address1" = '2451 fillingim street'
# """
# df_repaired_6 = pd.read_sql(q_6, conn)

In [ ]:
# df_repaired_6